# Technical Factor - Coefficient of Volatility (CVLT)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        m_ta_ema(high - low, 10) AS EC,
        (EC - m_lag(EC, 10)) / m_lag(EC, 10) * 100 AS CVLT,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(CVLT > 0 AND m_lag(CVLT, 1) <= 0, 1, 0) AS TRBY1,
        IF(CVLT < 0 AND m_lag(CVLT, 1) >= 0, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(CVLT > 0 AND (CVLT - m_lag(CVLT, 1)) > 0, 1, 0) AS MTBY1,
        IF(CVLT < 0 AND (CVLT - m_lag(CVLT, 1)) < 0, 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(CVLT < -20 AND (CVLT - m_lag(CVLT, 1)) > 0, 1, 0) AS RVBY1,
        IF(CVLT > 20 AND (CVLT - m_lag(CVLT, 1)) < 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(CVLT, 1) <= 10 AND CVLT > 10, 1, 0) AS BKBY1,
        IF(m_lag(CVLT, 1) >= -10 AND CVLT < -10, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        EC,
        CVLT,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,EC,CVLT,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-02-06,600519.SH,210.418081,-1.268468,0,0,0,0,0,0,0,0
1,2025-02-07,600519.SH,230.322050,-1.820410,0,0,0,1,0,0,0,0
2,2025-02-10,600519.SH,207.565987,-2.821596,0,0,0,1,0,0,0,0
3,2025-02-11,600519.SH,199.537074,-8.836764,0,0,0,1,0,0,0,0
4,2025-02-12,600519.SH,204.905425,0.315494,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
219,2025-12-25,600519.SH,113.124189,-12.842746,0,0,0,0,0,0,0,0
220,2025-12-26,600519.SH,106.593520,-13.657927,0,0,0,1,0,0,0,0
221,2025-12-29,600519.SH,107.378153,-12.547270,0,0,0,0,0,0,0,0
222,2025-12-30,600519.SH,112.274339,-7.417129,0,0,0,0,0,0,0,0
